<h2> 1. Importing required libraries and dataset </h2>

In [ ]:
! pip install librosa

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

%matplotlib inline
from matplotlib import pyplot as plt

import IPython.display as ipd

import os
import os.path
from os import path
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import time

from tqdm import tqdm
from tqdm.notebook import tqdm


import librosa 
import librosa.display
#Librosa is a python package for music and audio analysis.

import tensorflow as tf

In [ ]:
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

filenames = find_csv_filenames("/kaggle/input/rfcx-species-audio-detection")
for name in filenames:
  print(name)

In [ ]:
traindf=pd.read_csv(r"/kaggle/input/rfcx-species-audio-detection/train_tp.csv")

In [ ]:
traindf.shape

In [ ]:
traindf.head(10)

In [ ]:
print(f"total Species in Dataset : {len(traindf.recording_id)}")
print(f"total Recordings in Dataset : {len(traindf.recording_id.unique())}")

Creating an Index column as there is redundancy in Recording Id Column, May need a unique identifier

In [ ]:
traindf['idx'] = range(1, len(traindf) + 1)
traindf.head(5)

<h2>2. Data Analysis </h2>

#### Let's analyse the number of Species ID we have and their frequencies using count plot 

In [ ]:
sns.countplot(traindf['species_id'])

#### Species_id is balanced, all the categories have sufficient number of entries
Let's look into other features

In [ ]:
sns.countplot(traindf['songtype_id'])

In [ ]:
temp=traindf.loc[(traindf['songtype_id'] ==4)]
temp.head(10)

In [ ]:
path.exists("/kaggle/input/rfcx-species-audio-detection/train/0099c367b.flac")

In [ ]:
## playing audio for songtype_id=4
start = time.perf_counter()
data1, sr1 = librosa.load('/kaggle/input/rfcx-species-audio-detection/train/0099c367b.flac')
print("Time taken - " + str(time.perf_counter()-start))

In [ ]:
ipd.Audio('/kaggle/input/rfcx-species-audio-detection/train/003bec244.flac')

In [ ]:
path='/kaggle/input/rfcx-species-audio-detection/train/003bec244.flac'
y, sr = librosa.load(path)
plt.figure(figsize=(20,5))
librosa.display.waveplot(y, sr=sr)

In [ ]:
#display Spectrogram
path='/kaggle/input/rfcx-species-audio-detection/train/003bec244.flac'
x, sr = librosa.load(path)
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(20, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
#If to pring log of frequencies  
#librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

<h2>3. Data Preparation </h2>

* Creating Train and Validation split
* Triming the audio
* Reading the audio files with labels

In [ ]:
# Getting list of all filenames and shuffling them
IDX=np.random.permutation(traindf['idx'])
num_samples = len(IDX)
print('Number of total examples:', num_samples)

In [ ]:
train_idx = IDX[:850]
val_idx = IDX[850: 1216]
print('Training set size', len(train_idx))
print('Validation set size', len(val_idx))

In [ ]:
def trim_audio(IDX):
    recording_id=traindf.loc[traindf['idx'] == IDX,'recording_id'].iloc[0]
    t_min=traindf.loc[traindf['idx'] == IDX,'t_min'].iloc[0]
    t_max=traindf.loc[traindf['idx'] == IDX,'t_max'].iloc[0]
    label=traindf.loc[traindf['idx'] == IDX,'species_id'].iloc[0]
    #print(f"recording_id == {recording_id} & t_min = {t_min} & t_max = {t_max}")
    path='/kaggle/input/rfcx-species-audio-detection/train/'+recording_id+'.flac'
    #print(f"Path == {path}")
    y, sr = librosa.load(path)
    time_start = t_min*sr
    time_end = t_max*sr
    # Positioning sound slice
    # taking 5 seconds around center of the start and end time and cropping it accordingly
    effective_length=10*sr
    center = np.round((time_start + time_end) / 2)
    beginning = center - effective_length / 2
    if beginning < 0:
        beginning = 0
    beginning = np.random.randint(beginning , center)
    ending = beginning + effective_length
    if ending > len(y):
        ending = len(y)
    beginning = ending - effective_length
    y = y[beginning:ending].astype(np.float32)
    
    beginning_time = beginning / sr
    ending_time = ending / sr
    
    #query_string = f"recording_id == {recording_id} & "
    #query_string += f"t_min = {beginning_time} & t_max = {ending_time}"
    #print(query_string)
    #y=tf.squeeze(y, axis=-1)
    return y,label

Creating train and validation datasets

In [ ]:
traindf_new = pd.DataFrame(columns = ['audio', 'label'])
valdf_new=pd.DataFrame(columns = ['audio', 'label'])

In [ ]:
for index in tqdm(train_idx):
    d,l=trim_audio(index)
    traindf_new = traindf_new.append({'audio' : d, 'label' : l},ignore_index = True)

In [ ]:
traindf_new.head(10)

In [ ]:
def convert_tolist(x):
    return x.tolist()

In [ ]:
traindf_new.audio=traindf_new.audio.apply(convert_tolist)

In [ ]:
traindf_new.to_pickle('TrainWaveForm.csv')

In [ ]:
ipd.Audio(traindf_new.iloc[5].audio, rate=16000)

In [ ]:
for index in tqdm(val_idx):
    d,l=trim_audio(index)
    valdf_new = valdf_new.append({'audio' : d, 'label' : l},ignore_index = True) 

In [ ]:
valdf_new.audio=valdf_new.audio.apply(convert_tolist)

In [ ]:
valdf_new.to_pickle('ValWaveForm.csv')

### The output of this notebook is being utilised and code is continued in [TF Deep Soundscape Detection using Librosa Part-2](https://www.kaggle.com/krsna540/tf-deep-soundscape-detection-using-librosa-part-2)

### Helpful Resources
<hr/>

* https://www.youtube.com/watch?v=iCwMQJnKk2c&list=PL-wATfeyAMNqIee7cH3q1bh4QJFAaeNv0
* https://www.kaggle.com/c/rfcx-species-audio-detection/discussion/200922
* https://medium.com/@patrickbfuller/librosa-a-python-audio-libary-60014eeaccfb 
* https://towardsdatascience.com/extract-features-of-music-75a3f9bc265d
* https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html
* https://www.tensorflow.org/tutorials/audio/simple_audio